<a href="https://colab.research.google.com/github/ATUL27-03-2008/my-first--project-AI-ML-/blob/main/fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install numpy pandas scikit-learn nltk streamlit pyngrok



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import nltk
import re
import pickle

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

nltk.download('stopwords')

data = pd.read_csv("/content/fake_news_dataset.csv", encoding='latin1', on_bad_lines='skip', engine='python')

def clean_text(text):
    # Convert non-string types to string before processing
    text = str(text)
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [w for w in words if w not in stopwords.words('english')]
    return " ".join(words)

data['text'] = data['text'].apply(clean_text)

X = data['text']
y = data['label']

vectorizer = TfidfVectorizer(max_features=5000)
X_vec = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_vec, y, test_size=0.2, random_state=42
)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

pickle.dump(model, open("model.pkl", "wb"))
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Accuracy: 0.865546218487395


In [3]:
%%writefile app.py
import streamlit as st
import pickle
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

nltk.download('stopwords')
nltk.download('punkt')

model = pickle.load(open("model.pkl", "rb"))
vectorizer = pickle.load(open("vectorizer.pkl", "rb"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [w for w in words if w not in stopwords.words('english')]
    return " ".join(words)

def predict_news(news):
    cleaned = clean_text(news)
    vector = vectorizer.transform([cleaned])
    return model.predict(vector)[0]

def summarize_text(text, max_sentences=3):
    sentences = sent_tokenize(text)
    return " ".join(sentences[:max_sentences])

st.title("📰 Fake News Detector for Students")
news = st.text_area("Enter News Article")

if st.button("Check News"):
    result = predict_news(news)
    summary = summarize_text(news)

    if result == "FAKE":
        st.error("❌ This news is FAKE")
    else:
        st.success("✅ This news is REAL")

    st.write("Summary:")
    st.write(summary)


Writing app.py


In [4]:
!streamlit run app.py &>/content/logs.txt &


In [5]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
public_url


ERROR:pyngrok.process.ngrok:t=2026-01-17T03:07:47+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2026-01-17T03:07:47+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2026-01-17T03:07:47+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [9]:
from google.colab import userdata
from pyngrok import ngrok

# Retrieve the ngrok authtoken from Colab Secrets
# Make sure you have added your NGROK_AUTH_TOKEN in Colab Secrets
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

# Set the ngrok authtoken
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Re-establish the ngrok connection
public_url = ngrok.connect(8501)
print(f"Ngrok Tunnel URL: {public_url}")

Ngrok Tunnel URL: NgrokTunnel: "https://nonpacifiable-officially-hayes.ngrok-free.dev" -> "http://localhost:8501"


In [10]:
!git --version


git version 2.34.1


In [ ]:
!git config --global